In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


necessary libraries

In [2]:
# Install necessary libraries
!pip install mat73

import mat73
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


**Define dataset paths**


In [3]:
train_dataset_path = '/content/drive/MyDrive/train_32x32.mat'
test_dataset_path = '/content/drive/MyDrive/test_32x32.mat'


 **Load data function**

In [4]:
def load_data(path):
    """ Helper function for loading a MAT-File"""
    data = loadmat(path)  # Since the original dataset isn't using mat73 but loadmat
    return data['X'], data['y']


**Load training and test data**

In [5]:
X_train, y_train = load_data(train_dataset_path)
X_test, y_test = load_data(test_dataset_path)


Inspect the shape


In [6]:
print("Training Set", X_train.shape, y_train.shape)
print("Test Set", X_test.shape, y_test.shape)

Training Set (32, 32, 3, 73257) (73257, 1)
Test Set (32, 32, 3, 26032) (26032, 1)


**Transpose the dataset: (width, height, channels, samples) **

In [7]:
X_train = np.transpose(X_train, (3, 0, 1, 2))
X_test = np.transpose(X_test, (3, 0, 1, 2))

print("Reshaped Training Set", X_train.shape, y_train.shape)
print("Reshaped Test Set", X_test.shape, y_test.shape)

Reshaped Training Set (73257, 32, 32, 3) (73257, 1)
Reshaped Test Set (26032, 32, 32, 3) (26032, 1)


 Normalize the image data

In [8]:
X_train = X_train.astype('float32')/ 255.0
X_test = X_test.astype('float32')/ 255.0

Handle label 10

In [9]:
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

Convert the labels to one-hot encoding


In [10]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Split the training data into training and validation sets


In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training Set", X_train.shape, y_train.shape)
print("Validation Set", X_val.shape, y_val.shape)
print("Test Set", X_test.shape, y_test.shape)

Training Set (58605, 32, 32, 3) (58605, 10)
Validation Set (14652, 32, 32, 3) (14652, 10)
Test Set (26032, 32, 32, 3) (26032, 10)


 Building the CNN Model


In [12]:
def build_model():
    model = Sequential()

    # First convolution layer
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second convolution layer
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third convolution layer
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the output
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
     # Output layer (10 digits, softmax activation for classification)
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


Build the model


In [13]:
model = build_model()

# Print model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 160,202 (625.79 KB)

 Trainable params: 160,202 (625.79 KB)

 Non-trainable params: 0 (0.00 B)

Training the Model


In [14]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 108s 115ms/step - accuracy: 0.4391 - loss: 1.6385 - val_accuracy: 0.8319 - val_loss: 0.5728
Epoch 2/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 139s 112ms/step - accuracy: 0.8088 - loss: 0.6387 - val_accuracy: 0.8674 - val_loss: 0.4685
Epoch 3/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 132s 102ms/step - accuracy: 0.8513 - loss: 0.5049 - val_accuracy: 0.8825 - val_loss: 0.3960
Epoch 4/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 144s 104ms/step - accuracy: 0.8674 - loss: 0.4462 - val_accuracy: 0.8919 - val_loss: 0.3667
Epoch 5/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 93s 101ms/step - accuracy: 0.8801 - loss: 0.4060 - val_accuracy: 0.8959 - val_loss: 0.3538
Epoch 6/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 151s 110ms/step - accuracy: 0.8929 - loss: 0.3687 - val_accuracy: 0.8969 - val_loss: 0.3543
Epoch 7/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 105s 114ms/step - accuracy: 0.8969 - loss: 0.3520 - val_accuracy: 0.9016 - val_loss: 0.3392
Epoch 8/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 111s 121ms/step - accuracy: 0.9008 - 

 Evaluating the Model


In [15]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc * 100:.2f}%')


814/814 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.9078 - loss: 0.3832
Test accuracy: 91.10%


Plotting Training History


In [ ]:
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

 WRITE CODE TO SHOW PICTURE WITH PREDICTED AND ORIGBAL


In [ ]:
def pred_plot_with_original(model, x_test, y_test):
  """
  Predicts the class of the input image and plots it with the predicted class and the original label.

  Args:
    model: The trained CNN model.
    x_test: The input image data.
    y_test: The original labels of the images.
  """
  y_pred = model.predict(x_test)
  y_pred_classes = np.argmax(y_pred, axis=1)
  y_test_classes = np.argmax(y_test, axis=1)

  # Plot the image and predicted class
  plt.figure(figsize=(10, 10))
  for i in range(25):  # Display the first 25 images
      plt.subplot(5, 5, i + 1)
      plt.imshow(x_test[i])
      plt.title(f"Pred: {y_pred_classes[i]}, Orig: {y_test_classes[i]}")
      plt.axis('off')
  plt.show()


Save the model


In [ ]:
model.save('svhn_cnn_model.h5')

from google.colab import files
files.download('svhn_cnn_model.h5')


# UI function for user input and prediction
def predict_from_user_input():
    from google.colab import files
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))

        # Process the uploaded image (you might need to adjust this based on the image format)
        from PIL import Image
        import io
        image = Image.open(io.BytesIO(uploaded[fn])).convert('RGB')
        image = image.resize((32, 32))
        image_array = np.array(image) / 255.0  # Normalize
        image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model.predict(image_array)
        predicted_class = np.argmax(prediction)

        print(f"Predicted digit: {predicted_class}")

# Call the UI function to enable user input and prediction
predict_from_user_input()